In [ ]:
import pandas as pd

df_base = pd.read_excel("../fetched_data/형법.xlsx")

df_base

### JSONL 파일 생성

In [ ]:
import pandas as pd
import json

def df_to_jsonl(df : pd.DataFrame, path : str):
    if not path.endswith(".jsonl"):
        raise ValueError("a File name must ends with .jsonl")

    dataset = list()
    frame = {
        "id" : 0,
        "query": {
            "id": "",
            "part": 0,
            "chapter": 0,
            "content": "",
        },
    }

    for i, row in df.iterrows():
        if row["content"].strip().endswith("삭제"):
            continue
        # 조
        article = (
            f"제{row['article']}조"
            if row["article-branch"] == 1
            else f"제{row['article']}조의{row['article-branch']}"
        )
        # 항
        paragraph = f"제{row['paragraph']}항" if row["paragraph"] != 0 else ""
        # 호
        subparagraph = f"제{row['subparagraph']}호" if row["subparagraph"] != 0 else ""

        str_id = article + paragraph + subparagraph  # string
        data = frame.copy()
        data["query"] = {
            "id": str_id,
            "part" : row['part'], # 편
            "chapter": row["chapter"], # 장
            "content": row["content"],
        }
        dataset.append(data)

    for id, data in enumerate(dataset):
        data['id'] = id

    # JSONL 파일로 저장
    with open(path, 'w', encoding='utf-8') as f:
        for item in dataset:
            json.dump(item, f, ensure_ascii=False)
            f.write('\n')

VER.2  
KLAID 데이터셋 맞춤

In [ ]:
import pandas as pd
import json

def df_to_jsonl(df : pd.DataFrame, path : str, law_kind: str):
    if not path.endswith(".jsonl"):
        raise ValueError("a File name must ends with .jsonl")

    dataset = list()
    
    frame = {
        "id" : 0,
        "query": {
            "title": "",
            "content": "",
        },
    }

    for i, row in df.iterrows():
        # 삭제된 조항은 PASS
        if row["content"].strip().endswith("삭제"):
            continue
        # 조
        article = (
            f"제{row['article']}조"
            if row["article-branch"] == 1
            else f"제{row['article']}조의{row['article-branch']}"
        )
        # 항 
        paragraph = f" 제{row['paragraph']}항" if row["paragraph"] != 0 else ""
        # 호
        subparagraph = f" 제{row['subparagraph']}호" if row["subparagraph"] != 0 else ""

        str_id = f"{law_kind} {article}{paragraph}{subparagraph}"  # string
        data = frame.copy()
        data['query']['title'] = str_id
        data['query']['content'] = row['content']
        dataset.append(data)

    for id, data in enumerate(dataset):
        data['id'] = id

    # JSONL 파일로 저장
    with open(path, 'w', encoding='utf-8') as f:
        for item in dataset:
            json.dump(item, f, ensure_ascii=False)
            f.write('\n')

### 편/장 제목 (preamble == 1) 은 제외

In [2]:
df_base = pd.read_excel("../fetched_data/형법.xlsx")

df = df_base.loc[df_base['preamble'] == 0]

df = df.drop('preamble', axis=1)

# df_to_jsonl(df, "../data/law_queries.jsonl")
df_to_jsonl(df, "../data/형법.jsonl", law_kind="형법")

### 파일 양식 수정

In [2]:
from datasets import load_dataset

fpath = "../data/queries-edited.jsonl"
ds = load_dataset('json', data_files=fpath)['train']
ds

Generating train split: 0 examples [00:00, ? examples/s]

Dataset({
    features: ['id', 'query'],
    num_rows: 762
})

In [6]:
with open("../data/queries-edited-new.jsonl", "w", encoding="utf-8") as f:
    for data in ds:
        item = {
            "id": data["id"],
            "query": {
                "title": data["query"]["id"],
                "content": data["query"]["content"],
            },
        }
        json.dump(item, f, ensure_ascii=False)
        f.write("\n")

## GPT 문장 생성으로 query-pos 튜닝 데이터셋 구축

### Synchronous API 요청  

경험상 약 762개 행, pos 문장 3개씩 생성 -> 약 60분정도 소요  

In [ ]:
from openai import OpenAI
from pydantic import BaseModel
import pandas as pd
import json
import os
import tqdm


class Triplet(BaseModel):
    query: str
    pos: list[str]

def dataset_to_df(dataset):
    df = pd.DataFrame(dataset)

    df['id'] = df['query'].apply(lambda x : x['id'])
    df['content'] = df['query'].apply(lambda x : x['content'])

    df.drop('query', axis=1, inplace=True)
    
    return df

def generate_sentences(k : int, client, dataset: pd.DataFrame, dir_path: str):
    print(f"# total : {len(dataset)} #")
    path = os.path.join(dir_path, f"relevant_incidents.jsonl")
    responses = []
    with open("gpt_prompt.txt", "r", encoding="utf-8") as f:
        instructions = f.read().replace("#K#", str(k))

    total = len(dataset)
    for i, row in tqdm(dataset.iterrows(), total=total, desc="Generating"):
        prompt = f"{row['id']} # {row['content']}"

        try:
            res = client.responses.parse(
                model="gpt-4o-mini",
                instructions=instructions,
                input=[{"role": "user", "content": prompt}],
                text_format=Triplet
            )
            responses.append(dict(res.output_parsed))

        except Exception as e:
            print(f"Error processing row {i} ({row['id']}): {e}")
            continue  # 오류가 나면 그 행은 스킵하고 계속 진행

    with open(path, 'w', encoding='utf-8') as f:
        for res in responses:
            json.dump(res, f, ensure_ascii=False)
            f.write('\n')

In [ ]:
from datasets import load_dataset

dataset = load_dataset("json", data_files="../data/ft_data_queries.jsonl")['train']
df = dataset_to_df(dataset)

df

In [ ]:
from dotenv import load_dotenv

load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
client = OpenAI(OPENAI_API_KEY)
generate_sentences(5, client, df, f"../data/gpt_response")

### Asynchronous API 요청

In [3]:
from openai import AsyncOpenAI
from pydantic import BaseModel
import pandas as pd
import json
import os
from tqdm.asyncio import tqdm_asyncio
import asyncio

class Triplet(BaseModel):
    id : int
    title : str
    query: str
    pos: list[str]

def dataset_to_df(dataset):
    df = pd.DataFrame(dataset)

    df['title'] = df['query'].apply(lambda x : x['title'])
    df['content'] = df['query'].apply(lambda x : x['content'])

    df.drop('query', axis=1, inplace=True)
    
    return df

async def process_row_async(row, instructions, client):
    id = row['id']
    title = row['title']
    query = row['content']
    prompt = f"\"id\" : {id}, \"title\" : {title}, \"query\" : \"{query}\""

    try:
        response = await client.responses.parse(
            model='gpt-4.1',
            instructions=instructions,
            input=[{"role": "user", "content": prompt}],
            text_format=Triplet,
        )
        return dict(response.output_parsed)
    except Exception as e:
        print(f"Error in row {title} : {e}")
        return {'id': id , 'title': title, 'query': query, 'pos': []}
    
async def generate_sentences_async(instrctions, client, dataset : pd.DataFrame, dir_path: str, concurrency: int = 6):
    sem = asyncio.Semaphore(concurrency) # API 동시 요청 수 제한

    async def sem_task(row):
        async with sem:
            return await process_row_async(row, instrctions, client)
        
    tasks = [sem_task(row) for _, row in dataset.iterrows()]
    results = []

    for coro in tqdm_asyncio.as_completed(tasks, total=len(tasks), desc="Processing"):
        result = await coro
        if result:
            results.append(result)

    os.makedirs(dir_path, exist_ok=True)
    path = os.path.join(dir_path, "relevant_incidents.jsonl")

    # 비동기 처리 결과값들 원래 쿼리 데이터 순서대로 정렬
    sorted_results = sorted(results, key=lambda x : x['id'])

    with open(path, 'w', encoding='utf-8') as f:
        for res in sorted_results:
            json.dump(res, f, ensure_ascii=False)
            f.write('\n')

In [4]:
from datasets import load_dataset
import nest_asyncio
import asyncio

client = AsyncOpenAI(
    api_key=os.environ.get("OPENAI_API_KEY")
)

dataset = load_dataset("json", data_files="../data/queries-edited-filtered.jsonl")['train']
ds_df = dataset_to_df(dataset)

ds_df[:30]

,id,title,content
0,219,제87조,제87조(내란) 대한민국 영토의 전부 또는 일부에서 국가권력을 배제하거나 국헌을 문...
1,220,제87조제1호,우두머리는 사형 무기징역 또는 무기금고에 처한다
2,221,제87조제2호,모의에 참여하거나 지휘하거나 그 밖의 중요한 임무에 종사한 자는 사형 무기 또는 5...
3,222,제87조제3호,부화수행(附和隨行)하거나 단순히 폭동에만 관여한 자는 5년 이하의 징역이나 금고에 처한다
4,223,제88조,제88조(내란목적의 살인) 대한민국 영토의 전부 또는 일부에서 국가권력을 배제하거나...
5,229,제91조제1호,헌법 또는 법률에 정한 절차에 의하지 아니하고 헌법 또는 법률의 기능을 소멸시키는 것
6,230,제91조제2호,헌법에 의하여 설치된 국가기관을 강압에 의하여 전복 또는 그 권능행사를 불가능하게 ...
7,231,제92조,제92조(외환유치) 외국과 통모하여 대한민국에 대하여 전단을 열게 하거나 외국인과 ...
8,232,제93조,제93조(여적) 적국과 합세하여 대한민국에 항적한 자는 사형에 처한다
9,234,제94조제1항,적국을 위하여 모병한 자는 사형 또는 무기징역에 처한다


In [5]:
# gpt-4o-mini 약 6분 소요, 0.14 $
# gpt-4.1 약 15분, 2 $

k = 5
with open("gpt_prompt.txt", "r", encoding="utf-8") as f:
    instructions = f.read().replace("#K#", str(k))

nest_asyncio.apply() # jupyter notebook 자체적인 running event loop 가 존재하므로 실행 중 루프 (주피터 노트북) 내 중첩된 루프를 허용

asyncio.run(generate_sentences_async(instructions, client, ds_df, dir_path="../data/gpt_output"))

Processing: 100%|██████████| 363/363 [06:37<00:00,  1.09s/it]


### prompt test

In [23]:
def dataset_to_df(dataset):
    df = pd.DataFrame(dataset)

    df['id'] = df['query'].apply(lambda x : x['id'])
    df['content'] = df['query'].apply(lambda x : x['content'])

    df.drop('query', axis=1, inplace=True)
    
    return df

def mock_process_row(row):
    id = row['id']
    title = row['str_id']
    query = row['content']
    prompt = f"\"id\" : {id}, \"title\" : {title}, \"query\" : \"{query}\""

    print(prompt)

dataset = load_dataset("json", data_files="../data/queries.jsonl")['train']
ds_df = dataset_to_df(dataset)
# ds_df = ds_df.loc[(ds_df['id'] >= 751) & (ds_df['id'] <= 761)]

ds_df

# for i, data in ds_df.iterrows():
#     # print(data)
#     mock_process_row(data)

,id,content
0,제1조,제1조(범죄의 성립과 처벌)
1,제1조제1항,범죄의 성립과 처벌은 행위 시의 법률에 따른다
2,제1조제2항,범죄 후 법률이 변경되어 그 행위가 범죄를 구성하지 아니하게 되거나 형이 구법(舊法...
3,제1조제3항,재판이 확정된 후 법률이 변경되어 그 행위가 범죄를 구성하지 아니하게 된 경우에는 ...
4,제2조,제2조(국내범) 본법은 대한민국영역내에서 죄를 범한 내국인과 외국인에게 적용한다
...,...,...
757,제369조제1항,단체 또는 다중의 위력을 보이거나 위험한 물건을 휴대하여 제366조의 죄를 범한 때...
758,제369조제2항,제1항의 방법으로 제367조의 죄를 범한 때에는 1년 이상의 유기징역 또는 2천만원...
759,제370조,제370조(경계침범) 경계표를 손괴 이동 또는 제거하거나 기타 방법으로 토지의 경계...
760,제371조,제371조(미수범) 제366조 제367조와 제369조의 미수범은 처벌한다


In [ ]:
{
    "laws_service": "형법 제324조의5",
    "content": "제324조의5(미수범) 제324조 내지 제324조의4의 미수범은 처벌한다",
}

In [13]:
dataset = load_dataset("json", data_files="../data/queries-edited-filtered.jsonl")['train']
ds_df = dataset_to_df(dataset)
ds_df = ds_df.loc[(ds_df['id'] >= 751) & (ds_df['id'] <= 761)]

ds_df

,id,title,content
355,751,제366조,제366조(재물손괴등) 타인의 재물 문서 또는 전자기록등 특수매체기록을 손괴 또는 ...
356,752,제367조,제367조(공익건조물파괴) 공익에 공하는 건조물을 파괴한 자는 10년 이하의 징역 ...
357,754,제368조제1항,재물손괴죄를 범하여 사람의 생명 또는 신체에 대하여 위험을 발생하게 한 때에는 1년...
358,755,제368조제2항,재물손괴죄 또는 중손괴의 죄를 범하여 사람을 상해에 이르게 한 때에는 1년 이상의 ...
359,757,제369조제1항,단체 또는 다중의 위력을 보이거나 위험한 물건을 휴대하여 재물손괴죄를 범한 때에는 ...
360,758,제369조제2항,단체 또는 다중의 위력을 보이거나 위험한 물건을 휴대하여 중손괴죄를 범한 때에는 1...
361,759,제370조,제370조(경계침범) 경계표를 손괴 이동 또는 제거하거나 기타 방법으로 토지의 경계...
362,761,제372조,제372조(동력) 손괴의 죄에 관한 본장에서 관리할 수 있는 동력은 재물로 간주한다


### 파일 포맷 사후 수정

In [17]:
from datasets import load_dataset

dataset = load_dataset("json", data_files="../data/ft_data_queries.jsonl")['train']

str_ids = [query['id'] for query in dataset['query']]

ds_gpt = load_dataset('json', data_files="../data/relevant_incidents_gpt4.1_(2).jsonl")['train']

ds_gpt

Dataset({
    features: ['id', 'query', 'pos'],
    num_rows: 762
})

In [21]:
import json

ds_gpt_strID = ds_gpt.add_column('str_id', str_ids)

# 'str_id'을 원하는 위치로 재배치 (예: 맨 앞)
new_column_order = ['id', 'str_id', 'query', 'pos']

# 재정렬된 데이터셋 만들기
ds_gpt_strID = ds_gpt_strID.select_columns(new_column_order)

ds_gpt_strID.to_pandas()

fname = "../data/relevant_incidents_gpt4.1_(2).jsonl"
with open(fname, 'w', encoding='utf-8') as f:
    for res in ds_gpt_strID:
        json.dump(res, f, ensure_ascii=False)
        f.write('\n')